анализ холдеров

In [ ]:
import requests

def get_token_contract(ticker, api_key_coinmarketcap):
    url = f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/map"
    headers = {"X-CMC_PRO_API_KEY": api_key_coinmarketcap}
    params = {"symbol": ticker}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    if "data" in data:
        for token in data["data"]:
            print(f"Тикер: {token['symbol']}, Название: {token['name']}, ID: {token['id']}")
            details_url = f"https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?id={token['id']}"
            details = requests.get(details_url, headers=headers).json()
            if "contract_address" in details["data"][str(token["id"])]:
                print("Контракт:", details["data"][str(token["id"])]["contract_address"])
            else:
                print("Контракт не найден (возможно, это монета, а не токен)")
    else:
        print("Токен не найден")

API_KEY_CMC = ""
get_token_contract("CAKE", API_KEY_CMC)

Тикер: CAKE, Название: PancakeSwap, ID: 7186
Контракт: [{'contract_address': '0x152649eA73beAb28c5b49B26eb48f7EAD6d4c898', 'platform': {'name': 'Ethereum', 'coin': {'id': '1027', 'name': 'Ethereum', 'symbol': 'ETH', 'slug': 'ethereum'}}}, {'contract_address': '0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82', 'platform': {'name': 'BNB Smart Chain (BEP20)', 'coin': {'id': '1839', 'name': 'BNB', 'symbol': 'BNB', 'slug': 'bnb'}}}, {'contract_address': '0x1b896893dfc86bb67Cf57767298b9073D2c1bA2c', 'platform': {'name': 'Arbitrum', 'coin': {'id': '11841', 'name': 'Arbitrum', 'symbol': 'ARB', 'slug': 'arbitrum'}}}, {'contract_address': '0x159df6b7689437016108a019fd5bef736bac692b6d4a1f10c941f6fbb9a74ca6::oft::CakeOFT', 'platform': {'name': 'Aptos', 'coin': {'id': '21794', 'name': 'Aptos', 'symbol': 'APT', 'slug': 'aptos'}}}, {'contract_address': '0x3A287a06c66f9E95a56327185cA2BDF5f031cEcD', 'platform': {'name': 'zkSync Era', 'coin': {'id': '24091', 'name': 'ZKsync', 'symbol': 'ZK', 'slug': 'zksync'

In [ ]:
import requests
import pandas as pd

BSCSCAN_API_KEY = ""
TOKEN_CONTRACT = "0x524bC91Dc82d6b90EF29F76A3ECAaBAffFD490Bc"  # Адрес BEP-20 токена

url = f"https://api.bscscan.com/api?module=token&action=tokenholderlist&contractaddress={TOKEN_CONTRACT}&page=1&offset=100&apikey={BSCSCAN_API_KEY}"

response = requests.get(url)
data = response.json()

if data["status"] == "1":
    holders = data["result"]
    df = pd.DataFrame(holders)
    df["balance"] = df["value"].astype(float) / 1e18  # Конвертация из wei
    print(df.head(10))
else:
    print("Ошибка:", data["message"])

Ошибка: NOTOK


In [ ]:
import requests

TOKEN_ADDRESS = "0xbb4CdB9CBd36B01bD1cBaEBF2De08d9173bc095c"  # WBNB (пример)
CHAIN_ID = "bsc"  # Поддерживаемые сети: eth, bsc, polygon, arbitrum, celo и др.
ACCESS_KEY = ""

url = f"https://pro-openapi.debank.com/v1/token/top_holders?chain_id={CHAIN_ID}&id={TOKEN_ADDRESS}&start=0&limit=100"
headers = {
    "accept": "application/json",
    "AccessKey": ACCESS_KEY
}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    holders = response.json()["data"]["holders"]
    for holder in holders:
        print(f"Адрес: {holder['address']}, Баланс: {holder['amount']}")
else:
    print(f"Ошибка: {response.status_code}", response.json())

Ошибка: 403 {'message': 'Requests are limited, because of insufficient units, you can go to https://cloud.debank.com to recharge.'}


ликвидность

In [ ]:
import requests

TOKEN_ADDRESS = "0xd36932143f6ebdedd872d5fb0651f4b72fd15a84"  # Адрес токена на BSC
url = f"https://api.pancakeswap.info/api/v2/tokens/{TOKEN_ADDRESS}"
response = requests.get(url).json()

if "data" in response:
    liquidity = float(response["data"]["liquidity"])
    print(f"Ликвидность: ${liquidity:,.2f}")
    print("Риск скама:", "Высокий" if liquidity < 10000 else "Низкий")
else:
    print("Токен не найден в PancakeSwap!")

Токен не найден в PancakeSwap!


In [ ]:
response

{'error': {'code': 500,
  'message': 'Network error: Unexpected token p in JSON at position 4'}}

проверка смарт-контракта

In [ ]:
BSCSCAN_API_KEY = ""
CONTRACT_ADDRESS = "0x0fffe17bdb4fed64440da33606e32a84beb24f4a"  # Адрес BEP-20 токена

url = f"https://api.bscscan.com/api?module=contract&action=getabi&address={CONTRACT_ADDRESS}&apikey={BSCSCAN_API_KEY}"
response = requests.get(url).json()

if response["status"] == "1":
    print("Контракт верифицирован. Проверяем код на скам-функции...")
    # Анализ ABI (например, поиск mint)
    abi = response["result"]
    if "mint" in abi.lower():
        print("⚠️ Найдена функция mint – возможен неограниченный выпуск токенов!")
    if "blacklist" in abi.lower():
        print("⚠️ Найдена функция blacklist – возможен неограниченный выпуск токенов!")
    if "setFee" in abi.lower():
        print("⚠️ Найдена функция setFee – возможен неограниченный выпуск токенов!")
    if "withdraw" in abi.lower():
        print("⚠️ Найдена функция withdraw – возможен неограниченный выпуск токенов!")
    if "unlockLiquidity" in abi.lower():
        print("⚠️ Найдена функция unlockLiquidity – возможен неограниченный выпуск токенов!")
else:
    print("Контракт НЕ верифицирован – 99% скам!")

Контракт НЕ верифицирован – 99% скам!


In [ ]:
import requests

ETHERSCAN_API_KEY = ""
CONTRACT_ADDRESS = "0x0fffe17bdb4fed64440da33606e32a84beb24f4a"

def check_etherscan_contract(contract_address):
    url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url).json()

    if response["status"] == "1" and response["result"] != "Contract source code not verified":
        print("Контракт верифицирован. Проверяем код на опасные функции...")
        abi = response["result"].lower()

        scam_flags = {
            "mint": "⚠️ Найдена функция mint - возможен неограниченный выпуск токенов!",
            "blacklist": "⚠️ Найдена функция blacklist - разработчик может блокировать кошельки!",
            "setfee": "⚠️ Найдена функция setFee - могут изменять комиссии произвольно!",
            "withdraw": "⚠️ Найдена функция withdraw - могут выводить средства из контракта!",
            "unlock": "⚠️ Найдена функция unlock - могут разблокировать ликвидность!",
            "pause": "⚠️ Найдена функция pause - могут останавливать транзакции!",
            "changefee": "⚠️ Найдена функция changeFee - могут менять комиссии!",
            "owner": "⚠️ Найдены функции owner - возможен централизованный контроль!"
        }

        found_issues = False
        for func, warning in scam_flags.items():
            if func in abi:
                print(warning)
                found_issues = True

        if not found_issues:
            print("✅ Опасных функций не обнаружено (но это не гарантия безопасности)")
    else:
        print("❌ Контракт НЕ верифицирован - повышенный риск скама!")

    contract_url = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract_address}&apikey={ETHERSCAN_API_KEY}"
    contract_info = requests.get(contract_url).json()

    if contract_info["status"] == "1":
        print("\nДополнительная информация:")
        print(f"Имя контракта: {contract_info['result'][0]['ContractName']}")
        print(f"Компилятор: {contract_info['result'][0]['CompilerVersion']}")
        print(f"Оптимизация: {contract_info['result'][0]['OptimizationUsed']}")

# Запуск проверки
check_etherscan_contract(CONTRACT_ADDRESS)

Контракт верифицирован. Проверяем код на опасные функции...
⚠️ Найдены функции owner - возможен централизованный контроль!

Дополнительная информация:
Имя контракта: dydxProtocol
Компилятор: v0.6.0+commit.26b70077
Оптимизация: 1


In [ ]:
import requests

# Популярные EVM-сети и их публичные RPC-узлы
NETWORKS = {
    "Ethereum Mainnet": "https://rpc.ankr.com/eth",
    "Polygon": "https://rpc.ankr.com/polygon",
    "Binance Smart Chain": "https://bsc-dataseed.binance.org",
    "Arbitrum One": "https://arb1.arbitrum.io/rpc",
    "Optimism": "https://mainnet.optimism.io",
    "Avalanche C-Chain": "https://api.avax.network/ext/bc/C/rpc",
    "Fantom Opera": "https://rpcapi.fantom.network",
    # Можно добавить другие RPC
}

def get_contract_code(rpc_url, address):
    payload = {
        "jsonrpc": "2.0",
        "method": "eth_getCode",
        "params": [address, "latest"],
        "id": 1
    }
    try:
        response = requests.post(rpc_url, json=payload, timeout=5)
        result = response.json().get("result", "")
        return result
    except Exception as e:
        print(f"Ошибка при обращении к {rpc_url}: {e}")
        return None

def find_contract_networks(address):
    address = address.lower()
    if not address.startswith("0x") or len(address) != 42:
        raise ValueError("Некорректный адрес контракта")

    found_networks = []
    for name, rpc in NETWORKS.items():
        code = get_contract_code(rpc, address)
        if code and code != "0x":
            found_networks.append(name)
    return found_networks

if __name__ == "__main__":
    contract_address = input("Введите адрес контракта: ").strip()
    networks = find_contract_networks(contract_address)
    if networks:
        print("Контракт найден в следующих сетях:")
        for net in networks:
            print(f"- {net}")
    else:
        print("Контракт не найден ни в одной из поддерживаемых сетей.")


Введите адрес контракта: 0xd2877702675e6ceb975b4a1dff9fb7baf4c91ea9
Контракт не найден ни в одной из поддерживаемых сетей.


объем

In [ ]:
import requests

TOKEN_ID = "shiba-inu"
VS_CURRENCY = "usd"

try:
    url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency={VS_CURRENCY}&ids={TOKEN_ID}"
    response = requests.get(url, timeout=10)  # Таймаут 10 секунд
    response.raise_for_status()  # Проверка на HTTP-ошибки

    data = response.json()

    if not data:
        print("Токен не найден или нет данных.")
    else:
        volume = data[0]["total_volume"]
        print(f"Объем торгов {TOKEN_ID.upper()}: ${volume:,.2f}")
        print("Риск:", "Высокий" if volume < 10000 else "Низкий")

except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе к API: {e}")
except (KeyError, IndexError):
    print("Невозможно получить данные объема торгов.")
except Exception as e:
    print(f"Неожиданная ошибка: {e}")

Объем торгов SHIBA-INU: $127,078,166.00
Риск: Низкий


In [ ]:
import requests

TOKEN_ID = "kucoin-shares"
VS_CURRENCY = "usd"

try:
    url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency={VS_CURRENCY}&ids={TOKEN_ID}"
    response = requests.get(url, timeout=10)  # Таймаут 10 секунд
    response.raise_for_status()  # Проверка на HTTP-ошибки

    data = response.json()

    if not data:
        print("Токен не найден или нет данных.")
    else:
        volume = data[0]["total_volume"]
        print(f"Объем торгов {TOKEN_ID.upper()}: ${volume:,.2f}")
        print("Риск:", "Высокий" if volume < 10000 else "Низкий")

except requests.exceptions.RequestException as e:
    print(f"Ошибка при запросе к API: {e}")
except (KeyError, IndexError):
    print("Невозможно получить данные объема торгов.")
except Exception as e:
    print(f"Неожиданная ошибка: {e}")

Объем торгов KUCOIN-SHARES: $53,881.00
Риск: Низкий


In [ ]:
import requests

API_KEY = ""
CONTRACT_ADDRESS = "0xfa14fa6958401314851a17d6c5360ca29f74b57b"

url = f"https://pro-api.coinmarketcap.com/v2/cryptocurrency/info?address={CONTRACT_ADDRESS}"
headers = {
    "X-CMC_PRO_API_KEY": API_KEY,
    "Accept": "application/json"
}

response = requests.get(url, headers=headers)
data = response.json()

if "data" in data:
    for platform, info in data["data"].items():
        print(f"Сеть: {platform}")
        print(f"Название: {info['name']}")
        print(f"Тикер: {info['symbol']}")
        print(f"ID токена: {info['id']}")  # Для других запросов CMC
else:
    print("Токен не найден или ошибка API.")

Сеть: 9194
Название: Saito
Тикер: SAITO
ID токена: 9194


In [ ]:
data

{'status': {'timestamp': '2025-04-18T10:48:44.104Z',
  'error_code': 400,
  'error_message': 'No items found',
  'elapsed': 0,
  'credit_count': 0,
  'notice': None}}

## сбор данных

In [ ]:
import os
import requests
from datetime import datetime
from bs4 import BeautifulSoup

def check_audit_certik(project_name: str) -> bool:
    """Проверяет наличие аудита на CertiK."""
    url = f"https://www.certik.com/projects/{project_name.lower()}"
    try:
        return "Audit Report" in requests.get(url, timeout=5).text
    except:
        return False

def check_kyc_website(project_url: str) -> bool:
    """Проверяет KYC на сайте проекта."""
    if not project_url.startswith("http"):
        project_url = f"https://{project_url}"
    try:
        text = requests.get(project_url, timeout=5).text.lower()
        return any(word in text for word in ["kyc", "verified", "audit"])
    except:
        return False

def get_token_data(contract_address: str, blockchain: str = "ethereum") -> dict:
    """Основная функция сбора данных."""
    if blockchain == "ethereum":
        API_URL = "https://api.etherscan.io/api"
        API_KEY = ""
    elif blockchain == "bsc":
        API_URL = "https://api.bscscan.com/api"
        API_KEY = ""
    else:
        raise ValueError("Неподдерживаемый блокчейн. Используйте 'ethereum' или 'bsc'.")

    # Получаем данные контракта
    contract_source = requests.get(
        API_URL,
        params={
            "module": "contract",
            "action": "getsourcecode",
            "address": contract_address,
            "apikey": API_KEY,
        }
    ).json()

    is_verified = contract_source["result"][0]["SourceCode"] != ""
    contract_name = contract_source["result"][0]["ContractName"]

    # Дату создания контракта
    tx_list = requests.get(
        API_URL,
        params={
            "module": "account",
            "action": "txlist",
            "address": contract_address,
            "startblock": 0,
            "endblock": 99999999,
            "sort": "asc",
            "apikey": API_KEY,
        }
    ).json()

    if not tx_list["result"]:
        raise ValueError("Контракт не найден.")

    creation_time = datetime.fromtimestamp(int(tx_list["result"][0]["timeStamp"]))
    age_days = (datetime.now() - creation_time).days

    # Проверяем аудит и KYC
    has_audit = check_audit_certik(contract_name)
    has_kyc = check_kyc_website(f"{contract_name}.com")  # Пример: pancake.com

    return {
        "contract_address": contract_address,
        "blockchain": blockchain,
        "is_verified": is_verified,
        "age_days": age_days,
        "has_audit": has_audit,
        "has_kyc": has_kyc,
    }

# Пример использования
if __name__ == "__main__":
    # USDT на Ethereum
    print(get_token_data("0x0fffe17bdb4fed64440da33606e32a84beb24f4a", "ethereum"))

    # CAKE на BSC
    print(get_token_data("0x0fffe17bdb4fed64440da33606e32a84beb24f4a", "bsc"))

{'contract_address': '0x0fffe17bdb4fed64440da33606e32a84beb24f4a', 'blockchain': 'ethereum', 'is_verified': True, 'age_days': 1572, 'has_audit': False, 'has_kyc': False}


ValueError: Контракт не найден.

In [ ]:
import requests

# Настройки
ETHERSCAN_API_KEY = ""  # Бесплатно получите на etherscan.io
CONTRACT_ADDRESS = "0x0fffe17bdb4fed64440da33606e32a84beb24f4a"  # Адрес ERC-20 токена (USDT)

def check_etherscan_contract(contract_address):
    url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={ETHERSCAN_API_KEY}"
    response = requests.get(url).json()

    if response["status"] == "1" and response["result"] != "Contract source code not verified":
        print("Контракт верифицирован. Проверяем код на опасные функции...")
        abi = response["result"].lower()

        # Список опасных функций
        scam_flags = {
            "mint": "⚠️ Найдена функция mint - возможен неограниченный выпуск токенов!",
            "blacklist": "⚠️ Найдена функция blacklist - разработчик может блокировать кошельки!",
            "setfee": "⚠️ Найдена функция setFee - могут изменять комиссии произвольно!",
            "withdraw": "⚠️ Найдена функция withdraw - могут выводить средства из контракта!",
            "unlock": "⚠️ Найдена функция unlock - могут разблокировать ликвидность!",
            "pause": "⚠️ Найдена функция pause - могут останавливать транзакции!",
            "changefee": "⚠️ Найдена функция changeFee - могут менять комиссии!",
            "owner": "⚠️ Найдены функции owner - возможен централизованный контроль!"
        }

        found_issues = False
        for func, warning in scam_flags.items():
            if func in abi:
                print(warning)
                found_issues = True

        if not found_issues:
            print("✅ Опасных функций не обнаружено (но это не гарантия безопасности)")
    else:
        print("❌ Контракт НЕ верифицирован - повышенный риск скама!")

    # Дополнительная проверка через Etherscan API
    contract_url = f"https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract_address}&apikey={ETHERSCAN_API_KEY}"
    contract_info = requests.get(contract_url).json()

    if contract_info["status"] == "1":
        print("\nДополнительная информация:")
        print(f"Имя контракта: {contract_info['result'][0]['ContractName']}")
        print(f"Компилятор: {contract_info['result'][0]['CompilerVersion']}")
        print(f"Оптимизация: {contract_info['result'][0]['OptimizationUsed']}")

# Запуск проверки
check_etherscan_contract(CONTRACT_ADDRESS)

Контракт верифицирован. Проверяем код на опасные функции...
⚠️ Найдены функции owner - возможен централизованный контроль!

Дополнительная информация:
Имя контракта: dydxProtocol
Компилятор: v0.6.0+commit.26b70077
Оптимизация: 1


In [ ]:
BSCSCAN_API_KEY = ""  # Бесплатно получите на bscscan.com
CONTRACT_ADDRESS = "0x0fffe17bdb4fed64440da33606e32a84beb24f4a"  # Адрес BEP-20 токена

url = f"https://api.bscscan.com/api?module=contract&action=getabi&address={CONTRACT_ADDRESS}&apikey={BSCSCAN_API_KEY}"
response = requests.get(url).json()

if response["status"] == "1":
    print("Контракт верифицирован. Проверяем код на скам-функции...")
    # Анализ ABI (например, поиск mint)
    abi = response["result"]
    if "mint" in abi.lower():
        print("⚠️ Найдена функция mint – возможен неограниченный выпуск токенов!")
    if "blacklist" in abi.lower():
        print("⚠️ Найдена функция blacklist – возможен неограниченный выпуск токенов!")
    if "setFee" in abi.lower():
        print("⚠️ Найдена функция setFee – возможен неограниченный выпуск токенов!")
    if "withdraw" in abi.lower():
        print("⚠️ Найдена функция withdraw – возможен неограниченный выпуск токенов!")
    if "unlockLiquidity" in abi.lower():
        print("⚠️ Найдена функция unlockLiquidity – возможен неограниченный выпуск токенов!")
else:
    print("Контракт НЕ верифицирован – 99% скам!")

In [ ]:
import requests
import time
from datetime import datetime, timedelta

COINGECKO_API_KEY = "YOUR_API_KEY"

def get_dex_cex_data(token_address: str, blockchain: str = "ethereum"):
    """
    Анализ токена на DEX/CEX по 5 ключевым параметрам:
    1. Наличие листинга на CEX
    2. Объем торгов (24h)
    3. Изменение цены (24h/7d)
    4. Глубина ликвидности
    5. Крупные сливы
    """

    # Определяем ID блокчейна для CoinGecko
    blockchain_id = {
        "ethereum": "ethereum",
        "bsc": "binance-smart-chain"
    }.get(blockchain.lower(), "ethereum")

    # 1. Получаем базовые данные о токене
    base_url = "https://api.coingecko.com/api/v3"

    try:
        # Ищем токен по адресу контракта
        search_url = f"{base_url}/coins/{blockchain_id}/contract/{token_address}"
        token_data = requests.get(search_url).json()

        if "error" in token_data:
            return {"error": "Токен не найден на CoinGecko"}

        # 2. Проверяем листинг на CEX
        cex_listings = False
        tickers_url = f"{base_url}/coins/{token_data['id']}/tickers"
        tickers_data = requests.get(tickers_url).json()

        for ticker in tickers_data.get("tickers", []):
            if ticker["market"]["identifier"] in ["binance", "kraken", "coinbase", "huobi", "okex"]:
                cex_listings = True
                break

        # 3. Анализ объема и цены
        market_data = token_data.get("market_data", {})
        volume_24h = market_data.get("total_volume", {}).get("usd", 0)
        price_change_24h = market_data.get("price_change_percentage_24h", 0)
        price_change_7d = market_data.get("price_change_percentage_7d", 0)

        # 4. Глубина ликвидности (берем из DEX данных)
        liquidity = market_data.get("total_liquidity", {}).get("usd", 0)

        # 5. Поиск крупных сливов (анализ последних дней)
        large_dumps = False
        market_chart_url = f"{base_url}/coins/{token_data['id']}/market_chart?vs_currency=usd&days=7"
        chart_data = requests.get(market_chart_url).json()

        if len(chart_data["prices"]) > 1:
            price_changes = []
            for i in range(1, len(chart_data["prices"])):
                prev_price = chart_data["prices"][i-1][1]
                current_price = chart_data["prices"][i][1]
                change = (current_price - prev_price) / prev_price * 100
                price_changes.append(change)

            # Ищем падения >25% за 1 день
            large_dumps = any(change < -25 for change in price_changes)

        return {
            "cex_listings": cex_listings,
            "trading_volume_24h": volume_24h,
            "price_change_24h": price_change_24h,
            "price_change_7d": price_change_7d,
            "liquidity_usd": liquidity,
            "large_dumps_detected": large_dumps,
            "risk_score": calculate_risk_score(volume_24h, price_change_24h, liquidity, large_dumps)
        }

    except Exception as e:
        return {"error": str(e)}

def calculate_risk_score(volume, change_24h, liquidity, has_dumps):
    """Рассчитывает риск-скор на основе параметров"""
    score = 0

    # Объем торгов
    if volume < 10000: score += 3  # Очень низкий объем
    elif volume < 50000: score += 1

    # Изменение цены
    if abs(change_24h) > 30: score += 2  # Резкие скачки

    # Ликвидность
    if liquidity < 50000: score += 2  # Низкая ликвидность

    # Крупные сливы
    if has_dumps: score += 3

    return min(10, score)  # Максимальный риск 10/10

# Пример использования
if __name__ == "__main__":
    # Пример для Ethereum (USDT)
    usdt_data = get_dex_cex_data("0xdac17f958d2ee523a2206206994597c13d831ec7", "ethereum")
    print("Анализ USDT (Ethereum):")
    for k, v in usdt_data.items():
        print(f"{k}: {v}")

    # Пример для BSC (CAKE)
    cake_data = get_dex_cex_data("0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82", "bsc")
    print("\nАнализ CAKE (BSC):")
    for k, v in cake_data.items():
        print(f"{k}: {v}")

Анализ USDT (Ethereum):
cex_listings: True
trading_volume_24h: 16952754170
price_change_24h: 0.00628
price_change_7d: 0.03598
liquidity_usd: 0
large_dumps_detected: False
risk_score: 2

Анализ CAKE (BSC):
error: 'prices'


# GET DATA

In [ ]:
import requests
import sys

# API keys
ETHERSCAN_API_KEY = ''
COINGECKO_BASE = 'https://api.coingecko.com/api/v3'

def fetch_etherscan_abi(address: str):
    url = (f"https://api.etherscan.io/api?module=contract&action=getabi"
           f"&address={address}&apikey={ETHERSCAN_API_KEY}")
    return requests.get(url, timeout=10).json()

def fetch_etherscan_source(address: str):
    url = (f"https://api.etherscan.io/api?module=contract&action=getsourcecode"
           f"&address={address}&apikey={ETHERSCAN_API_KEY}")
    return requests.get(url, timeout=10).json()

def check_etherscan_contract(address: str) -> dict:
    """Returns verification status and ABI/sourceinfo"""
    abi_data = fetch_etherscan_abi(address)
    src_data = fetch_etherscan_source(address)

    res = {
        'is_verified': False,
        'has_mint': False,
        'has_blacklist': False,
        'has_setfee': False,
        'has_withdraw': False,
        'has_unlock': False,
        'has_pause': False,
        'has_changefee': False,
        'has_owner': False,
        'OptimizationUsed': ''
    }

    if abi_data.get('status') == '1' and abi_data.get('result') != 'Contract source code not verified':
        res['is_verified'] = True
        abi = abi_data['result'].lower()
        for flag in ['mint', 'blacklist', 'setfee', 'withdraw', 'unlock', 'pause', 'changefee', 'owner']:
            res[f'has_{flag}'] = (flag in abi)

    if src_data.get('status') == '1' and src_data.get('result'):
        info = src_data['result'][0]
        res['OptimizationUsed'] = info.get('OptimizationUsed', '')

    return res

def get_dex_cex_data(token_address: str) -> dict:
    """Returns market data from CoinGecko"""
    try:
        token_data = requests.get(
            f"{COINGECKO_BASE}/coins/ethereum/contract/{token_address}",
            timeout=10
        ).json()
    except:
        token_data = {}

    # Default values
    result = {
        'cex_listings': False,
        'trading_volume_24h': 0.0,
        'price_change_24h': 0.0,
        'price_change_7d': 0.0,
        'large_dumps_detected': False
    }

    cid = token_data.get('id')
    if not cid:
        return result

    # CEX listing
    try:
        tickers = requests.get(f"{COINGECKO_BASE}/coins/{cid}/tickers", timeout=10).json().get('tickers', [])
        result['cex_listings'] = any(t.get('market',{}).get('identifier') in ['binance','kraken','coinbase','huobi','okex']
                                  for t in tickers)
    except:
        pass

    # Market data
    md = token_data.get('market_data') or {}
    result['trading_volume_24h'] = float(md.get('total_volume', {}).get('usd', 0)) or 0.0
    result['price_change_24h'] = float(md.get('price_change_percentage_24h', 0)) or 0.0
    result['price_change_7d'] = float(md.get('price_change_percentage_7d', 0)) or 0.0

    # Large dumps detection
    try:
        chart = requests.get(f"{COINGECKO_BASE}/coins/{cid}/market_chart?vs_currency=usd&days=7", timeout=10).json()
        prices = chart.get('prices', [])
        for i in range(1, len(prices)):
            prev, curr = prices[i-1][1], prices[i][1]
            if prev and ((curr - prev)/prev*100) < -25:
                result['large_dumps_detected'] = True
                break
    except:
        pass

    return result

def get_token_info(contract_address: str):
    """Get all information about token by its contract address"""
    contract_info = check_etherscan_contract(contract_address)
    market_data = get_dex_cex_data(contract_address)

    # Combine and order the results exactly as requested
    ordered_results = {
        'is_verified': contract_info['is_verified'],
        'has_mint': contract_info['has_mint'],
        'has_blacklist': contract_info['has_blacklist'],
        'has_setfee': contract_info['has_setfee'],
        'has_withdraw': contract_info['has_withdraw'],
        'has_unlock': contract_info['has_unlock'],
        'has_pause': contract_info['has_pause'],
        'has_changefee': contract_info['has_changefee'],
        'has_owner': contract_info['has_owner'],
        'OptimizationUsed': contract_info['OptimizationUsed'],
        'cex_listings': market_data['cex_listings'],
        'trading_volume_24h': market_data['trading_volume_24h'],
        'price_change_24h': market_data['price_change_24h'],
        'price_change_7d': market_data['price_change_7d'],
        'large_dumps_detected': market_data['large_dumps_detected']
    }

    return ordered_results

if __name__ == '__main__':
    contract_address = '0x0a07525aa264a3e14cdbdd839b1eda02a34e2778'
    token_info = get_token_info(contract_address)
    print(token_info)

{'is_verified': True, 'has_mint': False, 'has_blacklist': False, 'has_setfee': False, 'has_withdraw': False, 'has_unlock': False, 'has_pause': False, 'has_changefee': False, 'has_owner': True, 'OptimizationUsed': '1', 'cex_listings': False, 'trading_volume_24h': 0.0, 'price_change_24h': 0.0, 'price_change_7d': 0.0, 'large_dumps_detected': False}


## XGBOOST

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('/content/dataset_with_features.csv')

In [ ]:
df

,contract_address,ticker,source,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,...,ContractName,CompilerVersion,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,liquidity_usd,large_dumps_detected,risk_score
0,0x01e1d7cbd3bc0eb1030485f33708421011459459,TOAD,trustwallet,not_scam,True,False,False,False,False,False,...,TOAD,v0.8.19+commit.7dd6d404,1.0,False,26415.0,-12.65765,1.41999,0.0,False,3.0
1,0xee6b3f5f90b8a37e78c31978192ed430ba906901,FCM,trapdoor_scam,scam,True,False,False,False,False,False,...,FloydCryptoMayweather,v0.5.17+commit.d19bba13,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x0a07525aa264a3e14cdbdd839b1eda02a34e2778,OFW,trustwallet,not_scam,True,False,False,False,False,False,...,OFW,v0.4.25+commit.59dbf8f1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x60b42779fb9404786089cccb7bc0f9bbead97e15,FRIENDSHIB,trapdoor_scam,scam,True,False,False,False,False,False,...,FriendlyShibaInu,v0.5.17+commit.d19bba13,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x36bf67c9aa1d67a06bfec145f08e20112a54c0f7,JNN,trustwallet,not_scam,True,False,False,False,False,False,...,JNN,v0.4.24+commit.e67f0147,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14709,0xf285d2b9dbbd97fa21c89084ea468e5c72705a87,FLOKIM,trapdoor_scam,scam,True,False,False,False,False,False,...,FlokiMeme,v0.5.0+commit.1d4f565a,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14710,0x82bdfb4c6f488fc47700cef12c448a2f13f8ff4f,SKT,trustwallet,not_scam,True,False,False,False,False,False,...,SealBlockToken,v0.4.23+commit.124ca40d,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14711,0x1b3511639cd0839b9b5ad4b981db8f7a8bb8cad7,WYVERN,trapdoor_scam,scam,True,False,False,False,False,False,...,TheIceElemental,v0.8.10+commit.fc410830,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14712,0xc5fb36dd2fb59d3b98deff88425a3f425ee469ed,TSUKA,trustwallet,not_scam,True,False,False,True,False,False,...,DejitaruTsuka,v0.8.9+commit.e5eed63a,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.drop(columns=['ContractName', 'CompilerVersion', 'ticker', 'contract_address', 'source'])

In [ ]:
df_dropped = df.dropna()

In [ ]:
df_dropped

,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,has_pause,has_changefee,has_owner,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,liquidity_usd,large_dumps_detected,risk_score
0,not_scam,True,False,False,False,False,False,False,False,True,1.0,False,26415.00,-12.65765,1.41999,0.0,False,3.0
12,not_scam,True,False,False,False,False,False,False,False,True,1.0,False,3004.29,-1.47952,-4.13863,0.0,False,5.0
14,not_scam,True,True,False,False,False,False,False,False,True,1.0,False,3655102.00,4.74840,-4.73710,0.0,False,2.0
20,not_scam,True,True,False,False,False,False,False,False,True,0.0,False,7.38,0.00000,1.38325,0.0,False,5.0
24,not_scam,True,True,False,False,False,False,True,False,True,0.0,False,124.43,0.00000,-6.32648,0.0,False,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14506,not_scam,True,False,False,False,False,False,False,False,False,1.0,False,0.00,5.15029,3.92315,0.0,False,5.0
14510,not_scam,True,False,True,False,False,False,False,False,True,1.0,False,87219.00,-13.80553,-28.80513,0.0,False,2.0
14566,not_scam,True,True,False,False,False,False,True,False,True,1.0,False,65431.00,0.17971,-1.50194,0.0,False,2.0
14578,not_scam,True,False,False,False,False,False,False,False,True,0.0,False,70096.00,-0.41888,-8.27187,0.0,False,2.0


In [ ]:
no_data = df[df['cex_listings'].isna()]

In [ ]:
data = pd.concat([df_dropped, no_data.sample(500)])

In [ ]:
data

,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,has_pause,has_changefee,has_owner,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,liquidity_usd,large_dumps_detected,risk_score
0,not_scam,True,False,False,False,False,False,False,False,True,1.0,False,26415.00,-12.65765,1.41999,0.0,False,3.0
12,not_scam,True,False,False,False,False,False,False,False,True,1.0,False,3004.29,-1.47952,-4.13863,0.0,False,5.0
14,not_scam,True,True,False,False,False,False,False,False,True,1.0,False,3655102.00,4.74840,-4.73710,0.0,False,2.0
20,not_scam,True,True,False,False,False,False,False,False,True,0.0,False,7.38,0.00000,1.38325,0.0,False,5.0
24,not_scam,True,True,False,False,False,False,True,False,True,0.0,False,124.43,0.00000,-6.32648,0.0,False,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,scam,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11349,scam,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2269,scam,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2358,not_scam,True,True,False,False,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data = data.fillna(0)

In [ ]:
label_encoder = LabelEncoder()
bool_cols = ['cex_listings', 'large_dumps_detected', 'is_verified', 'has_mint', 'has_blacklist',
             'has_setfee', 'has_withdraw', 'has_unlock', 'has_pause', 'has_changefee', 'has_owner']
data_dropped = data.drop(columns=['risk_score', 'liquidity_usd'])
data_dropped[bool_cols] = data_dropped[bool_cols].astype(int)
data_dropped['label'] = label_encoder.fit_transform(data_dropped['label'])

# Разделение на признаки и целевую переменную
X = data_dropped.drop('label', axis=1)
y = data_dropped['label']

# Преобразование булевых колонок в числовые
bool_cols = X.select_dtypes(include=['bool']).columns
X[bool_cols] = X[bool_cols].astype(int)

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели XGBoost
model = XGBClassifier(
    objective='binary:logistic',  # или 'multi:softmax' если классов больше 2
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)

# Предсказание и оценка модели
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8380952380952381

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.86      0.89       155
           1       0.67      0.76      0.71        55

    accuracy                           0.84       210
   macro avg       0.79      0.81      0.80       210
weighted avg       0.85      0.84      0.84       210



In [ ]:
import pickle

with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
data_dropped

,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,has_pause,has_changefee,has_owner,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,large_dumps_detected
0,0,1,0,0,0,0,0,0,0,1,1.0,0,26415.00,-12.65765,1.41999,0
12,0,1,0,0,0,0,0,0,0,1,1.0,0,3004.29,-1.47952,-4.13863,0
14,0,1,1,0,0,0,0,0,0,1,1.0,0,3655102.00,4.74840,-4.73710,0
20,0,1,1,0,0,0,0,0,0,1,0.0,0,7.38,0.00000,1.38325,0
24,0,1,1,0,0,0,0,1,0,1,0.0,0,124.43,0.00000,-6.32648,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9229,1,1,0,0,0,0,0,0,0,1,0.0,0,0.00,0.00000,0.00000,0
11349,1,1,0,0,0,0,0,0,0,1,0.0,0,0.00,0.00000,0.00000,0
2269,1,1,0,0,0,0,0,0,0,1,0.0,0,0.00,0.00000,0.00000,0
2358,0,1,1,0,0,0,0,0,0,1,1.0,0,0.00,0.00000,0.00000,0


In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [ ]:
y_test.value_counts()

,count
label,
0,96
1,14


## ВСЕ ОСТАЛЬНОЕ

In [ ]:
df

,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,has_pause,has_changefee,has_owner,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,liquidity_usd,large_dumps_detected,risk_score
0,not_scam,True,False,False,False,False,False,False,False,True,1.0,False,26415.0,-12.65765,1.41999,0.0,False,3.0
1,scam,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,not_scam,True,False,False,False,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,scam,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,not_scam,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14709,scam,True,False,False,False,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14710,not_scam,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14711,scam,True,False,False,False,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14712,not_scam,True,False,False,True,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df

,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,has_pause,has_changefee,has_owner,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,liquidity_usd,large_dumps_detected,risk_score
0,0,True,False,False,False,False,False,False,False,True,1.0,False,26415.0,-12.65765,1.41999,0.0,False,3.0
1,1,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,True,False,False,False,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14709,1,True,False,False,False,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14710,0,True,False,False,False,False,False,False,False,True,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14711,1,True,False,False,False,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14712,0,True,False,False,True,False,False,False,False,True,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
label_encoder = LabelEncoder()
bool_cols = ['is_verified', 'has_mint', 'has_blacklist',
             'has_setfee', 'has_withdraw', 'has_unlock', 'has_pause', 'has_changefee', 'has_owner']
df[bool_cols] = df[bool_cols].astype(int)
df['label'] = label_encoder.fit_transform(df['label'])

# Разделение на признаки и целевую переменную
X = df.drop(columns=['label', 'trading_volume_24h', 'price_change_24h', 'cex_listings', 'price_change_7d', 'liquidity_usd', 'large_dumps_detected', 'risk_score'])
y = df['label']

# Преобразование булевых колонок в числовые
bool_cols = X.select_dtypes(include=['bool']).columns
X[bool_cols] = X[bool_cols].astype(int)

# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели XGBoost
model = XGBClassifier(
    objective='binary:logistic',  # или 'multi:softmax' если классов больше 2
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train)

# Предсказание и оценка модели
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.6914712878015631

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.50      0.62      1470
           1       0.64      0.88      0.74      1473

    accuracy                           0.69      2943
   macro avg       0.72      0.69      0.68      2943
weighted avg       0.72      0.69      0.68      2943



In [ ]:
X_train

,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,has_pause,has_changefee,has_owner,OptimizationUsed
14280,1,1,0,0,0,0,1,0,1,1.0
10114,1,0,0,0,0,0,0,0,1,0.0
2303,1,1,0,0,0,0,0,0,1,1.0
7087,1,0,0,0,0,0,0,0,1,1.0
11973,1,0,0,0,0,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...
5191,1,0,0,0,0,0,0,0,1,1.0
13418,1,1,0,0,0,0,0,0,1,0.0
5390,1,0,0,0,0,0,0,0,1,0.0
860,1,1,0,0,0,0,0,0,1,1.0


In [ ]:
df

,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,has_pause,has_changefee,has_owner,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,liquidity_usd,large_dumps_detected,risk_score
0,0,1,0,0,0,0,0,0,0,1,1.0,False,26415.0,-12.65765,1.41999,0.0,False,3.0
1,1,1,0,0,0,0,0,0,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,1,0,0,0,0,0,0,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,0,0,0,0,0,0,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1,0,0,0,0,0,0,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14709,1,1,0,0,0,0,0,0,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14710,0,1,0,0,0,0,0,0,0,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14711,1,1,0,0,0,0,0,0,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14712,0,1,0,0,1,0,0,0,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
no_data = df[df['cex_listings'].isna()]

In [ ]:
lis = list(no_data['contract_address'])

In [ ]:
TOKENS_TO_ANALYZE = []
for i in lis:
  TOKENS_TO_ANALYZE.append({"contract": i, "blockchain": "ethereum"})
  TOKENS_TO_ANALYZE.append({"contract": i, "blockchain": "bsc"})

In [ ]:
import requests
import time
import csv
import json
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from typing import List, Dict, Optional

# --- Configuration ---
COINGECKO_API_KEY = "YOUR_API_KEY"  # Optional but recommended
MAX_THREADS = 5  # To avoid rate limits
REQUEST_DELAY = 1.5  # Seconds between requests (avoid rate limits)
OUTPUT_FORMAT = "csv"  # "csv" or "json"
OUTPUT_FILENAME = "token_analysis_report"


# --- API Helpers ---
def make_coingecko_request(url: str, params: Optional[Dict] = None) -> Dict:
    """Handles rate limits and API errors."""
    try:
        headers = {"x-cg-pro-api-key": COINGECKO_API_KEY} if COINGECKO_API_KEY else {}
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        time.sleep(REQUEST_DELAY)  # Avoid rate limits
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# --- Analysis Functions ---
def analyze_token(contract: str, blockchain: str = "ethereum") -> Dict:
    """Fetches and analyzes token data from CoinGecko."""
    blockchain_id = {
        "ethereum": "ethereum",
        "bsc": "binance-smart-chain",
        "polygon": "polygon-pos",
    }.get(blockchain.lower(), "ethereum")

    # 1. Fetch token details
    token_url = f"https://api.coingecko.com/api/v3/coins/{blockchain_id}/contract/{contract}"
    token_data = make_coingecko_request(token_url)

    if "error" in token_data:
        return {"contract": contract, "blockchain": blockchain, "error": token_data["error"]}

    # 2. Check CEX listings
    tickers_url = f"https://api.coingecko.com/api/v3/coins/{token_data['id']}/tickers"
    tickers_data = make_coingecko_request(tickers_url)
    cex_listings = any(
        ticker["market"]["identifier"] in ["binance", "coinbase", "kraken", "huobi", "okex"]
        for ticker in tickers_data.get("tickers", [])
    )

    # 3. Market data (volume, price changes, liquidity)
    market_data = token_data.get("market_data", {})
    volume_24h = market_data.get("total_volume", {}).get("usd", 0)
    price_change_24h = market_data.get("price_change_percentage_24h", 0)
    price_change_7d = market_data.get("price_change_percentage_7d", 0)
    liquidity = market_data.get("total_liquidity", {}).get("usd", 0)

    # 4. Detect large dumps (>25% drop in 24h over last 7 days)
    chart_url = f"https://api.coingecko.com/api/v3/coins/{token_data['id']}/market_chart?vs_currency=usd&days=7"
    chart_data = make_coingecko_request(chart_url)
    large_dumps = False
    if "prices" in chart_data and len(chart_data["prices"]) > 1:
        for i in range(1, len(chart_data["prices"])):
            prev_price = chart_data["prices"][i-1][1]
            current_price = chart_data["prices"][i][1]
            if (current_price - prev_price) / prev_price * 100 < -25:
                large_dumps = True
                break

    return {
        "contract": contract,
        "blockchain": blockchain,
        "name": token_data.
        get("name", "N/A"),
        "symbol": token_data.get("symbol", "N/A"),
        "cex_listings": cex_listings,
        "trading_volume_24h": volume_24h,
        "price_change_24h": price_change_24h,
        "price_change_7d": price_change_7d,
        "liquidity_usd": liquidity,
        "large_dumps_detected": large_dumps,
        "risk_score": calculate_risk_score(volume_24h, price_change_24h, liquidity, large_dumps),
        "coingecko_url": f"https://www.coingecko.com/en/coins/{token_data['id']}",
    }

def calculate_risk_score(volume: float, change_24h: float, liquidity: float, has_dumps: bool) -> int:
    """Calculates a risk score (0-10)."""
    score = 0
    if volume < 10000: score += 3
    elif volume < 50000: score += 1
    if abs(change_24h) > 30: score += 2
    if liquidity < 50000: score += 2
    if has_dumps: score += 3
    return min(10, score)

# --- Batch Processing ---
def analyze_all_tokens(tokens: List[Dict]) -> List[Dict]:
    """Processes multiple tokens in parallel."""
    results = []
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = [
            executor.submit(analyze_token, token["contract"], token["blockchain"])
            for token in tokens
        ]
        for future in futures:
            try:
                results.append(future.result())
            except Exception as e:
                print(f"Error processing token: {e}")
    return results

# --- Export Results ---
def save_results(data: List[Dict], format: str = "csv"):
    """Saves data to CSV or JSON."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{OUTPUT_FILENAME}_{timestamp}.{format}"

    if format == "csv":
        with open(filename, "w", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=data[0].keys())
            writer.writeheader()
            writer.writerows(data)
    elif format == "json":
        with open(filename, "w") as f:
            json.dump(data, f, indent=2)
    print(f"Report saved to {filename}")

# --- Main Execution ---
if __name__ == "__main__":
    print(f"Analyzing {len(TOKENS_TO_ANALYZE)} tokens...")
    results = analyze_all_tokens(TOKENS_TO_ANALYZE)
    save_results(results, OUTPUT_FORMAT)
    print("Analysis complete!")

Analyzing 28330 tokens...
Report saved to token_analysis_report_20250425_093237.csv
Analysis complete!


In [ ]:
pip install moralis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.6
    Uninstalling frozendict-2.4.6:
      Successfully uninstalled frozendict-2.4.6


In [ ]:
import requests
import pandas as pd
from moralis import evm_api
from concurrent.futures import ThreadPoolExecutor
import time

# --- Конфигурация ---
MORALIS_API_KEY = ""
MAX_THREADS = 3
OUTPUT_FILE = "crypto_scan_results.csv"

# Поддерживаемые сети (все возможные в Moralis + DEX)
SUPPORTED_CHAINS = [
    "eth",          # Ethereum
    "bsc",          # Binance Smart Chain
    "polygon",      # Polygon
    "avalanche",    # Avalanche C-Chain
    "fantom",       # Fantom Opera
    "arbitrum",     # Arbitrum
    "optimism",     # Optimism
    "cronos",       # Cronos
    "harmony",      # Harmony
    "moonbeam",     # Moonbeam
    "moonriver",    # Moonriver
    "celo",         # Celo
    "gnosis",       # Gnosis Chain
    "heco",         # HECO Chain
    "klaytn",       # Klaytn
    "metis",        # Metis
    "aurora",       # Aurora (NEAR)
    "oasis",        # Oasis Emerald
    "velas",        # Velas
    "iotex",        # IoTeX
    "telos",        # Telos EVM
    "kava"          # Kava EVM
]

# DEX API для проверки ликвидности (TheGraph)
DEX_API_URLS = {
    "eth": "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
    "bsc": "https://api.thegraph.com/subgraphs/name/pancakeswap/exchange-v2",
    "polygon": "https://api.thegraph.com/subgraphs/name/quickswap/quickswap",
    "avalanche": "https://api.thegraph.com/subgraphs/name/traderjoe-xyz/exchange",
    "fantom": "https://api.thegraph.com/subgraphs/name/spookyswap/exchange",
    "arbitrum": "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-arbitrum",
    "optimism": "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-optimism",
    "cronos": "https://api.thegraph.com/subgraphs/name/vvs-finance/vvs-exchange"
}

# --- Основные функции ---
def detect_chain(contract: str) -> str:
    """Определяет блокчейн контракта через Moralis."""
    for chain in SUPPORTED_CHAINS:
        try:
            result = evm_api.token.get_token_metadata(
                api_key=MORALIS_API_KEY,
                params={"addresses": [contract], "chain": chain},
            )
            if result and result[0].get("name"):
                return chain
        except:
            continue
    return "unknown"

def get_dex_liquidity(contract: str, chain: str) -> float:
    """Проверяет ликвидность на DEX через TheGraph."""
    if chain not in DEX_API_URLS:
        return 0.0

    query = f"""
    {{
        token(id: "{contract.lower()}") {{
            totalLiquidity
        }}
    }}
    """
    try:
        response = requests.post(
            DEX_API_URLS[chain],
            json={"query": query},
            timeout=10
        ).json()
        return float(response.get("data", {}).get("token", {}).get("totalLiquidity", 0))
    except:
        return 0.0

def get_token_info(contract: str) -> dict:
    """Собирает полные данные о токене."""
    chain = detect_chain(contract)
    if chain == "unknown":
        return {"contract": contract, "error": "Chain not detected"}

    try:
        # Данные через Moralis
        moralis_data = evm_api.token.get_token_metadata(
            api_key=MORALIS_API_KEY,
            params={"addresses": [contract], "chain": chain},
        )[0]

        # Проверка ликвидности
        liquidity = get_dex_liquidity(contract, chain)

        return {
            "contract": contract,
            "chain": chain,
            "name": moralis_data.get("name"),
            "symbol": moralis_data.get("symbol"),
            "decimals": moralis_data.get("decimals"),
            "dex_liquidity": liquidity,
            "is_active": liquidity > 0,
        }
    except Exception as e:
        return {"contract": contract, "chain": chain, "error": str(e)}

def batch_process_contracts(contracts: list) -> pd.DataFrame:
    """Параллельная обработка контрактов."""
    results = []
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        futures = [executor.submit(get_token_info, contract) for contract in contracts]
        for future in futures:
            try:
                results.append(future.result())
            except:
                continue
    return pd.DataFrame(results)

# --- Запуск ---
if __name__ == "__main__":
    # Загрузка контрактов из файла
    print(f"🔍 Scanning {len(lis)} contracts across {len(SUPPORTED_CHAINS)} chains...")
    df = batch_process_contracts(lis[:100])  # Первые 1000 для теста

    # Фильтрация и сохранение
    live_tokens = df[df["is_active"] == True]
    live_tokens.to_csv(OUTPUT_FILE, index=False)

    print(f"✅ Results saved to {OUTPUT_FILE}")
    print(f"💡 Active tokens found: {len(live_tokens)}")
    print(f"🌐 Chains detected: {df['chain'].unique().tolist()}")

🔍 Scanning 14165 contracts across 22 chains...
✅ Results saved to crypto_scan_results.csv
💡 Active tokens found: 0
🌐 Chains detected: ['eth', nan]


In [ ]:
df

,contract,chain,name,symbol,decimals,dex_liquidity,is_active,error
0,0xee6b3f5f90b8a37e78c31978192ed430ba906901,eth,Floyd Crypto Mayweather,FCM,8,0.0,False,NaN
1,0x0a07525aa264a3e14cdbdd839b1eda02a34e2778,eth,OFW,OFW,18,0.0,False,NaN
2,0x60b42779fb9404786089cccb7bc0f9bbead97e15,eth,Friendly Shiba Inu,FriendShib,18,0.0,False,NaN
3,0x36bf67c9aa1d67a06bfec145f08e20112a54c0f7,eth,JeyennCoin,JNN,18,0.0,False,NaN
4,0x80da04fba4be56cd2661af272ea93bfe835954fd,eth,ShibaExchange Inu,SHIBE,18,0.0,False,NaN
...,...,...,...,...,...,...,...,...
95,0x3d7dba86d21cc6a6a2e2e901d32128c2241cde8f,eth,swerve.fi,SWRV,18,0.0,False,NaN
96,0x5f53f7a8075614b699baad0bc2c899f4bad8fbbf,eth,Rebellious,REBL,18,0.0,False,NaN
97,0x05f80e9015e85d74c94922ab7c0367ec91a00577,eth,FEATHER,FEATHER,18,0.0,False,NaN
98,0x5d7f6540b8d592e81def42904c853a6dd01bd486,eth,Bizcoin,BIZ,18,0.0,False,NaN


In [ ]:
import requests

def get_dex_data(contract: str, chain: str = "ethereum"):
    """Fetches volume, price changes, and liquidity from DEX subgraphs."""
    subgraph_urls = {
        "ethereum": "https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2",
        "bsc": "https://api.thegraph.com/subgraphs/name/pancakeswap/exchange-v2",
        "polygon": "https://api.thegraph.com/subgraphs/name/quickswap/quickswap",
    }

    query = f"""
    {{
        token(id: "{contract.lower()}") {{
            tradeVolumeUSD
            totalLiquidity
            tokenDayData(first: 7, orderBy: date, orderDirection: desc) {{
                priceUSD
                date
            }}
        }}
    }}
    """

    try:
        response = requests.post(subgraph_urls[chain], json={"query": query}).json()
        data = response["data"]["token"]

        # Calculate 24h/7d price changes
        prices = [float(d["priceUSD"]) for d in data["tokenDayData"]]
        price_change_24h = ((prices[0] - prices[1]) / prices[1]) * 100 if len(prices) > 1 else 0
        price_change_7d = ((prices[0] - prices[-1]) / prices[-1]) * 100 if len(prices) >= 7 else 0

        return {
            "trading_volume_24h": float(data["tradeVolumeUSD"]),
            "price_change_24h": price_change_24h,
            "price_change_7d": price_change_7d,
            "liquidity_usd": float(data["totalLiquidity"])
        }
    except Exception as e:
        return {"error": str(e)}

res = []
for i in lis:
  res.append({"contract": i, "result": get_dex_data(i, "ethereum")})

In [ ]:
import requests
import time

# Replace with a token contract address (example is Shiba Inu on Ethereum)
CONTRACT_ADDRESS = "0x7311e2114d89ab5f85fe5013cd33fe07297c0fdb"  # Replace with your token
CHAIN = "ethereum"  # or 'bsc', 'polygon', etc.

def get_token_data(contract_address, chain='ethereum'):
    url = f"https://api.dexscreener.com/latest/dex/tokens/{contract_address}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error fetching data from DexScreener: {response.status_code}")
        return None

    data = response.json()
    print(data)
    if 'pairs' not in data or len(data['pairs']) == 0:
        print("No trading pairs found.")
        return None

    # Picking the most liquid pair
    most_liquid = max(data['pairs'], key=lambda x: float(x.get('liquidity', {}).get('usd', 0)))

    return {
        'name': most_liquid.get('baseToken', {}).get('name'),
        'symbol': most_liquid.get('baseToken', {}).get('symbol'),
        'price_usd': most_liquid.get('priceUsd'),
        'price_change_1h': most_liquid.get('priceChange', {}).get('h1'),
        'price_change_6h': most_liquid.get('priceChange', {}).get('h6'),
        'price_change_24h': most_liquid.get('priceChange', {}).get('h24'),
        'liquidity_usd': most_liquid.get('liquidity', {}).get('usd'),
        'volume_usd_24h': most_liquid.get('volume', {}).get('h24'),
        'fdv': most_liquid.get('fdv'),
        'pair_created_at': time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(most_liquid.get('pairCreatedAt', 0) / 1000)),
        'dex': most_liquid.get('dexId'),
        'url': most_liquid.get('url')
    }

def print_token_insights(data):
    print(f"\n📊 Token Information")
    print(f"Name: {data['name']} ({data['symbol']})")
    print(f"DEX: {data['dex']}")
    print(f"Price: ${float(data['price_usd']):,.8f}")
    print(f"Price Change: 1h: {data['price_change_1h']}%, 6h: {data['price_change_6h']}%, 24h: {data['price_change_24h']}%")
    print(f"Liquidity: ${float(data['liquidity_usd']):,.2f}")
    print(f"Volume (24h): ${float(data['volume_usd_24h']):,.2f}")
    print(f"FDV: ${float(data['fdv']) if data['fdv'] else 'N/A'}")
    print(f"Pair Created At: {data['pair_created_at']}")
    print(f"More Info: {data['url']}\n")

    # 🚨 Simple Heuristics for Scam Detection
    print("🚨 Scam Risk Analysis:")
    if float(data['liquidity_usd']) < 5000:
        print("⚠️  Very low liquidity! May be a rug pull or inactive token.")
    if float(data['volume_usd_24h']) < 100:
        print("⚠️  Very low trading volume. Token may be illiquid or dead.")
    if data['fdv'] and float(data['fdv']) > 1_000_000_000:
        print("⚠️  Extremely high FDV. Possibly overvalued or fake supply.")
    if '-' in str(data['price_change_24h']) and float(data['price_change_24h']) < -50:
        print("⚠️  Huge price drop in 24h. Possible dump.")
    if '+' in str(data['price_change_24h']) and float(data['price_change_24h']) > 1000:
        print("⚠️  Huge price increase. Could be manipulated.")
    print()

# Main Execution
token_info = get_token_data(CONTRACT_ADDRESS, CHAIN)
if token_info:
    print_token_insights(token_info)
else:
    print("Token data could not be retrieved.")

{'schemaVersion': '1.0.0', 'pairs': None}


TypeError: object of type 'NoneType' has no len()

In [ ]:
data

NameError: name 'data' is not defined

In [ ]:
df

,contract_address,ticker,source,label,is_verified,has_mint,has_blacklist,has_setfee,has_withdraw,has_unlock,...,ContractName,CompilerVersion,OptimizationUsed,cex_listings,trading_volume_24h,price_change_24h,price_change_7d,liquidity_usd,large_dumps_detected,risk_score
0,0x01e1d7cbd3bc0eb1030485f33708421011459459,TOAD,trustwallet,not_scam,True,False,False,False,False,False,...,TOAD,v0.8.19+commit.7dd6d404,1.0,False,26415.0,-12.65765,1.41999,0.0,False,3.0
1,0xee6b3f5f90b8a37e78c31978192ed430ba906901,FCM,trapdoor_scam,scam,True,False,False,False,False,False,...,FloydCryptoMayweather,v0.5.17+commit.d19bba13,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x0a07525aa264a3e14cdbdd839b1eda02a34e2778,OFW,trustwallet,not_scam,True,False,False,False,False,False,...,OFW,v0.4.25+commit.59dbf8f1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x60b42779fb9404786089cccb7bc0f9bbead97e15,FRIENDSHIB,trapdoor_scam,scam,True,False,False,False,False,False,...,FriendlyShibaInu,v0.5.17+commit.d19bba13,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x36bf67c9aa1d67a06bfec145f08e20112a54c0f7,JNN,trustwallet,not_scam,True,False,False,False,False,False,...,JNN,v0.4.24+commit.e67f0147,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14709,0xf285d2b9dbbd97fa21c89084ea468e5c72705a87,FLOKIM,trapdoor_scam,scam,True,False,False,False,False,False,...,FlokiMeme,v0.5.0+commit.1d4f565a,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14710,0x82bdfb4c6f488fc47700cef12c448a2f13f8ff4f,SKT,trustwallet,not_scam,True,False,False,False,False,False,...,SealBlockToken,v0.4.23+commit.124ca40d,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14711,0x1b3511639cd0839b9b5ad4b981db8f7a8bb8cad7,WYVERN,trapdoor_scam,scam,True,False,False,False,False,False,...,TheIceElemental,v0.8.10+commit.fc410830,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14712,0xc5fb36dd2fb59d3b98deff88425a3f425ee469ed,TSUKA,trustwallet,not_scam,True,False,False,True,False,False,...,DejitaruTsuka,v0.8.9+commit.e5eed63a,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
